# Covid Lung Image analysis

In [69]:
# Import Dependencies
import numpy
import pandas
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import random
import shutil
import cv2
import sklearn.metrics
from pathlib import Path

# Keras Stuff
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

Now that dependencies are installed, lets understand the data used.

In [70]:
p_covid = Path("./data/CT_COVID")
p_not_coivd = Path("./data/CT_NonCOVID")
p_cov_len = len([x for x in p_covid.iterdir()])
p_not_cov_len = len([x for x in p_not_coivd.iterdir()])
print(f"Scans Covid: {p_cov_len}\nScans with No Covid: {p_not_cov_len}")


Scans Covid: 349
Scans with No Covid: 397


Lets Split Data into test set and training set.

10% of data will be used to test, 90% will be used to train the model.

In [71]:
train_data_cov = int(0.9 * p_cov_len)
train_data_cov_no = int(0.9 * p_not_cov_len)
print(f"Train Data COVID: {train_data_cov}\nTest Data Covid: {p_cov_len - train_data_cov}\nTrain Data NO COVID: {train_data_cov_no}\nTest Data No Covid: {p_not_cov_len - train_data_cov_no}")

Train Data COVID: 314
Test Data Covid: 35
Train Data NO COVID: 357
Test Data No Covid: 40


In [72]:
# Move Test Files Away
os.makedirs("./train/covid",exist_ok=True)
os.makedirs("./train/notcovid",exist_ok=True)
os.makedirs("./test/covid",exist_ok=True)
os.makedirs("./test/notcovid",exist_ok=True)
print("Made Dirs")

Made Dirs


In [73]:
random.seed(456)
train_data_cov_to_move = random.sample([x for x in p_covid.iterdir()], train_data_cov)
file_name_train_cov = [str(f).split("\\")[2] for f in train_data_cov_to_move]
for i in range(len(train_data_cov_to_move)):
    shutil.copy(f"./data/CT_COVID/{file_name_train_cov[i]}",f"./train/covid/{file_name_train_cov[i]}")

print(f"Moved {len(file_name_train_cov)} train covid")

c = 0
to_mov_test_cov = [str(x).split("\\")[2] for x in p_covid.iterdir()]
for file in to_mov_test_cov:
    if file not in file_name_train_cov:
        file = file.replace("\\","/")
        shutil.copy(f"./data/CT_COVID/{file}", f"./test/covid/{file}")
        c += 1
print(f"Moved {c} test covid")

Moved 314 train covid
Moved 35 test covid


In [74]:
train_data_no_cov_to_move = random.sample([x for x in  p_not_coivd.iterdir()], train_data_cov_no)
file_name_train_no_cov = [str(f).split("\\")[2] for f in train_data_no_cov_to_move]
for i in range(len(train_data_cov_to_move)):
    shutil.copy(f"./data/CT_NonCOVID/{file_name_train_no_cov[i]}",f"./train/notcovid/{file_name_train_no_cov[i]}")

print(f"Moved {len(file_name_train_no_cov)} train nocovid")

c = 0
to_mov_test_no_cov = [str(x).split("\\")[2] for x in p_not_coivd.iterdir()]
for file in to_mov_test_no_cov:
    if file not in file_name_train_no_cov:
        shutil.copy(f"./data/CT_NonCOVID/{file}", f"./test/notcovid/{file}")
        c += 1
print(f"Moved {c} test nocovid")

Moved 357 train nocovid
Moved 40 test nocovid


Now the Data has been split

We Will train the data.

In [ ]:
# Some Config Vars
batch_size_number = 64
epoch = 10
h = 150
h = 150